# Mask R-CNN - Human pose estimation

Inspect and visualize data loading and pre-processing code.

In [1]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

import utils
import visualize
from visualize import display_images
import model as modellib
from model import log

%matplotlib inline 

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, "mylogs")
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco_humanpose.h5")



## Configurations

Run one of the code blocks below to import and load the configurations to use.

In [2]:
# MS COCO Dataset
import coco

class TrainingConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1 #changed to 1. GPU to small for 2 images

training_config = TrainingConfig()

COCO_DIR = "D:/Eigene Dateien/Dokumente/coco"  # TODO: enter your own path here

## Dataset

In [3]:
# Load dataset
assert training_config.NAME == "coco"
# Training dataset
# load person keypoints dataset
train_dataset_keypoints = coco.CocoDataset(task_type="person_keypoints")
train_dataset_keypoints.load_coco(COCO_DIR, "train")
train_dataset_keypoints.prepare()

#Validation dataset
val_dataset_keypoints = coco.CocoDataset(task_type="person_keypoints")
val_dataset_keypoints.load_coco(COCO_DIR, "val")
val_dataset_keypoints.prepare()

print("Train Keypoints Image Count: {}".format(len(train_dataset_keypoints.image_ids)))
print("Train Keypoints Class Count: {}".format(train_dataset_keypoints.num_classes))
for i, info in enumerate(train_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

print("Val Keypoints Image Count: {}".format(len(val_dataset_keypoints.image_ids)))
print("Val Keypoints Class Count: {}".format(val_dataset_keypoints.num_classes))
for i, info in enumerate(val_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

loading annotations into memory...
Done (t=9.34s)
creating index...
index created!
Skeleton: (19, 2)
Keypoint names: (17,)
loading annotations into memory...
Done (t=0.32s)
creating index...
index created!
Skeleton: (19, 2)
Keypoint names: (17,)
Train Keypoints Image Count: 64115
Train Keypoints Class Count: 2
  0. BG                                                
  1. person                                            
Val Keypoints Image Count: 2693
Val Keypoints Class Count: 2
  0. BG                                                
  1. person                                            


## Ceate Model

In [4]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="training", model_dir=MODEL_DIR, config=training_config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
print("Loading weights from ", COCO_MODEL_PATH)
# model.keras_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Loading weights from  C:\Users\LStue\Mask_RCNN_Humanpose\mask_rcnn_coco_humanpose.h5


In [ ]:
# Training - Stage 1 #15 Epochs
#print("Train heads")
#model.train(train_dataset_keypoints, val_dataset_keypoints,
#            learning_rate=training_config.LEARNING_RATE,
#            epochs=1,
#            layers='heads')
# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
#print("Training Resnet layer 4+")
#model.train(train_dataset_keypoints, val_dataset_keypoints,
#            learning_rate=training_config.LEARNING_RATE / 10,
#            epochs=20,
#            layers='4+')
# Training - Stage 3
# Finetune layers from ResNet stage 3 and up
print("Training Resnet layer 3+")
model.train(train_dataset_keypoints, val_dataset_keypoints,
            learning_rate=training_config.LEARNING_RATE / 100,
            epochs=10,
            layers='all')

Training Resnet layer 3+

Starting at epoch 0. LR=2e-05

Checkpoint Path: C:\Users\LStue\Mask_RCNN_Humanpose\mylogs\coco20201229T1313\mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_

D:\Users\LStue\anaconda3\envs\tf21\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


Epoch 1/100


D:\Users\LStue\anaconda3\envs\tf21\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
D:\Users\LStue\anaconda3\envs\tf21\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
D:\Users\LStue\anaconda3\envs\tf21\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown sha

   1/1000 [..............................] - ETA: 7:39:09 - loss: 9.0529 - rpn_class_loss: 1.2353e-04 - rpn_bbox_loss: 0.0831 - mrcnn_class_loss: 0.3221 - mrcnn_bbox_loss: 0.6195 - keypoint_mrcnn_mask_loss: 6.7324 - mrcnn_mask_loss: 1.2958

...

D:\Users\LStue\anaconda3\envs\tf21\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


 341/1000 [=========>....................] - ETA: 16:26 - loss: 5.5524 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2001 - mrcnn_class_loss: 0.2033 - mrcnn_bbox_loss: 0.4513 - keypoint_mrcnn_mask_loss: 4.2579 - mrcnn_mask_loss: 0.4315

D:\Users\LStue\anaconda3\envs\tf21\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)


 342/1000 [=========>....................] - ETA: 16:25 - loss: 5.5407 - rpn_class_loss: 0.0083 - rpn_bbox_loss: 0.2006 - mrcnn_class_loss: 0.2037 - mrcnn_bbox_loss: 0.4515 - keypoint_mrcnn_mask_loss: 4.2454 - mrcnn_mask_loss: 0.4312

D:\Users\LStue\anaconda3\envs\tf21\lib\site-packages\skimage\transform\_warps.py:830: FutureWarning: Input image dtype is bool. Interpolation is not defined with bool data type. Please set order to 0 or explicitely cast input image to another data type. Starting from version 0.19 a ValueError will be raised instead of this warning.
  order = _validate_interpolation_order(image.dtype, order)
